<a href="https://colab.research.google.com/github/pablotrocini/cash-flow-app/blob/main/Cash_Flow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
from google.colab import drive
drive.mount('/content/drive')

MessageError: Error: credential propagation was unsuccessful

You can upload files to your Colab environment in several ways:

1.  **Using the file icon in the left sidebar:** Click on the folder icon (Files) in the left sidebar. Then, click on the 'Upload to session storage' icon (up arrow) and select your files.
2.  **Mounting Google Drive:** You can mount your Google Drive to access files directly from there.
3.  **Using `files.upload()`:** This Python function provides a simple interface to upload files from your local machine.
4.  **Using `wget` or `curl`:** If your files are hosted online, you can download them directly into Colab.

Below is an example of how to upload files using `files.upload()`:

In [ ]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print(f'User uploaded file "{fn}" with length {len(uploaded[fn])} bytes')

In [ ]:
!pip install XlsxWriter

In [11]:
import pandas as pd
from datetime import datetime, timedelta

# ==========================================
# PARTE 1: PROCESAMIENTO DE DATOS (LÓGICA ANTERIOR)
# ==========================================

fecha_hoy = pd.to_datetime(datetime.now().date())
# fecha_hoy = pd.to_datetime('2025-12-02') # Descomentar para probar con fecha fija

archivo_proyeccion = 'Proyeccion Pagos.xlsx'
archivo_cheques = 'Cheques.xlsx'

# Data de correlación incrustada directamente
data_nombres = {
    'Cheques': [
        'BBVA FRANCES BYC', 'BBVA FRANCES MPZ', 'BBVA FRANCES MBZ', 'BBVA FRANCES MGX',
        'BBVA FRANCES RG2', 'CREDICOOP BYC', 'CREDICOOP MGX', 'CREDICOOP MBZ',
        'CREDICOOP TMX', 'DE LA NACION ARG. BYC', 'DE LA NACION ARG MGX',
        'PATAGONIA MBZ', 'SANTANDER RIO BYC', 'SANTANDER RIO MBZ',
        'SANTANDER MGXD', 'MERCADO PAGO BYC', 'MERCADO PAGO MGX', 'MERCADO PAGO MBZ'
    ],
    'Proyeccion Pagos': [
        'Bco BBVA BYC SA', 'Bco BBVA MPZ BYC SA', 'Bco BBVA MBZ SRL', 'Bco BBVA MGXD SRL',
        'Bco BBVA RG2', 'Bco Credicoop BYC SA', 'Bco Credicoop MGXD SRL', 'Bco Credicoop MBZ SRL',
        'Bco Credicoop TMX SRL', 'Bco Nacion BYC SA', 'Bco Nacion MGXD SRL',
        'Bco Patagonia MBZ SRL', 'Bco Santander BYC SA', 'Bco Santander MBZ SRL',
        'Bco Santander MGXD SRL', 'MercadoPago BYC', 'MercadoPago MGX', 'MercadoPago MBZ'
    ],
    'EMPRESA': [
        'BYC', 'BYC', 'MBZ', 'MGX', 'BYC', 'BYC', 'MGX', 'MBZ',
        'TMX', 'BYC', 'MGX', 'MBZ', 'BYC', 'MBZ',
        'MGX', 'BYC', 'MGX', 'MBZ'
    ]
}
nombres_df = pd.DataFrame(data_nombres)

def procesar_archivo(ruta, col_banco, col_fecha, col_importe, tipo_origen, nombres_map_df):
    df = pd.read_excel(ruta)
    df_clean = pd.DataFrame({
        'Banco_Raw': df.iloc[:, col_banco].astype(str).str.strip(),
        'Fecha': pd.to_datetime(df.iloc[:, col_fecha], errors='coerce'),
        'Importe': pd.to_numeric(df.iloc[:, col_importe], errors='coerce'),
        'Origen': tipo_origen
    })
    df_clean = df_clean.dropna(subset=['Importe', 'Banco_Raw'])

    # Prepare nombres_map_df for merging based on tipo_origen
    if tipo_origen == 'Proyeccion':
        merge_on_col = 'Proyeccion Pagos'
        nombres_map_df_cleaned = nombres_map_df[['Proyeccion Pagos', 'EMPRESA']].copy()
        nombres_map_df_cleaned['Proyeccion Pagos'] = nombres_map_df_cleaned['Proyeccion Pagos'].astype(str).str.strip()
    elif tipo_origen == 'Cheques':
        merge_on_col = 'Cheques'
        nombres_map_df_cleaned = nombres_map_df[['Cheques', 'EMPRESA']].copy()
        nombres_map_df_cleaned['Cheques'] = nombres_map_df_cleaned['Cheques'].astype(str).str.strip()
    else:
        # Fallback if tipo_origen is neither, though it shouldn't happen with current calls
        df_clean['Banco_Limpio'] = df_clean['Banco_Raw']
        df_clean['Empresa'] = 'UNKNOWN' # Or derive from Banco_Raw if a pattern exists
        return df_clean

    # Perform merge
    df_clean = pd.merge(
        df_clean,
        nombres_map_df_cleaned,
        left_on='Banco_Raw',
        right_on=merge_on_col,
        how='left'
    )

    # Assign mapped Banco_Limpio and Empresa
    df_clean['Banco_Limpio'] = df_clean[merge_on_col].fillna(df_clean['Banco_Raw'])
    df_clean['Empresa'] = df_clean['EMPRESA'].fillna('UNKNOWN') # Fallback for unmapped entries

    # Drop temporary merge columns
    df_clean = df_clean.drop(columns=[merge_on_col, 'EMPRESA'])

    return df_clean

# Cargar y unir
print("Leyendo archivos...")
df_proy = procesar_archivo(archivo_proyeccion, 0, 2, 9, 'Proyeccion', nombres_df)
df_cheq = procesar_archivo(archivo_cheques, 3, 1, 14, 'Cheques', nombres_df)
df_total = pd.concat([df_proy, df_cheq])

# Periodos
fecha_limite_semana = fecha_hoy + timedelta(days=5)

# 1. Vencido
filtro_vencido = df_total['Fecha'] < fecha_hoy
df_vencido = df_total[filtro_vencido].groupby(['Empresa', 'Banco_Limpio'])[['Importe']].sum()
df_vencido.columns = ['Vencido']

# 2. Semana (Días)
filtro_semana = (df_total['Fecha'] >= fecha_hoy) & (df_total['Fecha'] <= fecha_limite_semana)
df_semana_data = df_total[filtro_semana].copy()
dias_es = {0:'lun', 1:'mar', 2:'mié', 3:'jue', 4:'vie', 5:'sáb', 6:'dom'}
df_semana_data['Nombre_Dia'] = df_semana_data['Fecha'].apply(lambda x: f"{x.strftime('%d-%b')} {dias_es[x.weekday()]}")

df_semana_pivot = pd.pivot_table(
    df_semana_data, index=['Empresa', 'Banco_Limpio'], columns='Nombre_Dia', values='Importe', aggfunc='sum', fill_value=0
)
df_semana_pivot['Total Semana'] = df_semana_pivot.sum(axis=1)

# 3. Emitidos (Futuro solo cheques)
filtro_emitidos = (df_total['Fecha'] > fecha_limite_semana) & (df_total['Origen'] == 'Cheques')
df_emitidos = df_total[filtro_emitidos].groupby(['Empresa', 'Banco_Limpio'])[['Importe']].sum()
df_emitidos.columns = ['Emitidos']

# Unir todo
reporte_final = pd.concat([df_vencido, df_semana_pivot, df_emitidos], axis=1).fillna(0)

# Ordenar columnas
cols = list(reporte_final.columns)
col_vencido = ['Vencido'] if 'Vencido' in cols else []
col_emitidos = ['Emitidos'] if 'Emitidos' in cols else []
col_total_sem = ['Total Semana'] if 'Total Semana' in cols else []
col_dias = sorted([c for c in cols if c not in col_vencido + col_emitidos + col_total_sem])
orden_final = col_vencido + col_dias + col_total_sem + col_emitidos
reporte_final = reporte_final[orden_final]

# ==========================================
# PARTE 2: DISEÑO VISUAL (XLSXWRITER)
# ==========================================
print("Generando Excel con formato...")

nombre_archivo_salida = 'Resumen_Cashflow_Formateado.xlsx'
writer = pd.ExcelWriter(nombre_archivo_salida, engine='xlsxwriter')
workbook = writer.book
worksheet = workbook.add_worksheet('Resumen')

# --- DEFINICIÓN DE FORMATOS ---
# Color Naranja fuerte (Encabezados)
fmt_header = workbook.add_format({
    'bold': True, 'font_color': 'white', 'bg_color': '#ED7D31',
    'border': 1, 'align': 'center', 'valign': 'vcenter'
})
# Color Naranja suave (Subtotales)
fmt_subtotal = workbook.add_format({
    'bold': True, 'bg_color': '#FCE4D6', 'num_format': '$ #,##0',
    'border': 1
})
# Formato Moneda normal
fmt_currency = workbook.add_format({
    'num_format': '$ #,##0', 'border': 1
})
# Formato Texto Banco
fmt_text = workbook.add_format({'border': 1})

# --- ESCRIBIR ENCABEZADOS ---
worksheet.write('A1', 'Resumen Cashflow', workbook.add_format({'bold': True, 'font_size': 14}))
worksheet.write('A2', f"Fecha Actual: {fecha_hoy.strftime('%d/%m/%Y')}")

# Escribir la fila de títulos de columnas (Fila 4, índice 3)
fila_actual = 3
col_bancos = 0
worksheet.write(fila_actual, col_bancos, "Etiquetas de fila", fmt_header)

# Escribir los nombres de las columnas de datos
columnas_datos = reporte_final.columns.tolist()
for i, col_name in enumerate(columnas_datos):
    worksheet.write(fila_actual, i + 1, col_name, fmt_header)

fila_actual += 1

# --- ESCRIBIR DATOS POR GRUPO (EMPRESA) ---
# Obtenemos lista única de empresas del índice
empresas_unicas = reporte_final.index.get_level_values(0).unique()

for empresa in empresas_unicas:
    # Escribir cabecera de grupo (opcional, aquí lo saltamos y vamos directo a los bancos)
    # Filtramos los datos de esta empresa
    datos_empresa = reporte_final.loc[empresa]

    # Ensure datos_empresa is always a DataFrame for iteration over rows
    # If it's a Series (single bank for this company), convert it to a DataFrame
    # preserving the bank name as index.
    if isinstance(datos_empresa, pd.Series):
        banco_limpio_idx = datos_empresa.name[1] # Extract Banco_Limpio from the Series name (multi-index tuple)
        datos_empresa = pd.DataFrame(datos_empresa).T # Convert to DataFrame and transpose
        datos_empresa.index = [banco_limpio_idx] # Set the index to the Banco_Limpio name
        datos_empresa.index.name = 'Banco_Limpio' # Set the index name

    for banco, row in datos_empresa.iterrows():
        worksheet.write(fila_actual, 0, banco, fmt_text) # Nombre Banco

        for i, val in enumerate(row):
            worksheet.write(fila_actual, i + 1, val, fmt_currency)

        fila_actual += 1

    # --- CREAR FILA DE SUBTOTAL ---
    worksheet.write(fila_actual, 0, f"Total {empresa}", fmt_subtotal)

    # Calcular y escribir sumas
    sumas = datos_empresa.sum()
    for i, val in enumerate(sumas):
        worksheet.write(fila_actual, i + 1, val, fmt_subtotal)

    fila_actual += 1 # Espacio extra o siguiente linea

# Ajustar ancho de columnas
worksheet.set_column(0, 0, 25) # Columna Bancos ancha
worksheet.set_column(1, len(columnas_datos), 15) # Columnas de importes

writer.close()
print(f"¡Listo! Archivo generado: {nombre_archivo_salida}")

Leyendo archivos...
Generando Excel con formato...
¡Listo! Archivo generado: Resumen_Cashflow_Formateado.xlsx


In [10]:
print(nombres_df.to_string())

                  Cheques        Proyeccion Pagos EMPRESA
0        BBVA FRANCES BYC         Bco BBVA BYC SA     BYC
1        BBVA FRANCES MPZ     Bco BBVA MPZ BYC SA     BYC
2        BBVA FRANCES MBZ        Bco BBVA MBZ SRL     MBZ
3        BBVA FRANCES MGX       Bco BBVA MGXD SRL     MGX
4        BBVA FRANCES RG2            Bco BBVA RG2     BYC
5           CREDICOOP BYC    Bco Credicoop BYC SA     BYC
6           CREDICOOP MGX  Bco Credicoop MGXD SRL     MGX
7           CREDICOOP MBZ   Bco Credicoop MBZ SRL     MBZ
8           CREDICOOP TMX   Bco Credicoop TMX SRL     TMX
9   DE LA NACION ARG. BYC       Bco Nacion BYC SA     BYC
10   DE LA NACION ARG MGX     Bco Nacion MGXD SRL     MGX
11          PATAGONIA MBZ   Bco Patagonia MBZ SRL     MBZ
12      SANTANDER RIO BYC    Bco Santander BYC SA     BYC
13      SANTANDER RIO MBZ   Bco Santander MBZ SRL     MBZ
14         SANTANDER MGXD  Bco Santander MGXD SRL     MGX
15       MERCADO PAGO BYC         MercadoPago BYC     BYC
16       MERCA

In [8]:
nombres_df = pd.read_excel('Nombres.xlsx')
display(nombres_df.head())

,Cheques,Proyeccion Pagos,EMPRESA
0,BBVA FRANCES BYC,Bco BBVA BYC SA,BYC
1,BBVA FRANCES MPZ,Bco BBVA MPZ BYC SA,BYC
2,BBVA FRANCES MBZ,Bco BBVA MBZ SRL,MBZ
3,BBVA FRANCES MGX,Bco BBVA MGXD SRL,MGX
4,BBVA FRANCES RG2,Bco BBVA RG2,BYC
